# Fast DGSEM block inversion

In this notebook, we test the methods analytical formulations for DGSEM-matrix inversion against classical `numpy` tools. We also time them to see which is the most efficient one.

To test for exactness, we relying on a classical Frobenius norm of the differnce between the proposed analytical formulae and the reference one.

The performances will be measured with `jupyter` `%%timeit` magic (see the header of the related cells). To compute fair statistics, we use 100 runs (see option `-r 100`) and the default values for loops (10'000): hence, running the notebook might take some time.

In [1]:
import numpy as np

import fast_DGSEM_block_inversion as f_dgsem

In [2]:
# Polynomial order
p = 2  # Accepted: 1:5 included

# Celerity*time step over spatial grid size
lambda_x, lambda_y = 1.0, 1.0

## Exactness

In [3]:
_ = f_dgsem.compare_eigenvalues_computation(p)

Verification of L eigenvalues (difference inf. norm between the two methods): 2.4874942663908416e-15



In [4]:
_ = f_dgsem.compare_2D_inversion(p, lambda_x, lambda_y)

Verification of diag. block inv. (difference inf. norm between the two methods): 8.17216081584016e-15



In [5]:
_ = f_dgsem.compare_2D_inversion_viscosity(p, lambda_x, lambda_y)

Verification of diag. block inv. with graph visc. (difference inf. norm between the two methods): 2.891387197060117e-16


## Performances

Setting up needed variables

In [6]:
lobatto_weights = f_dgsem.LOBATTO_WEIGHTS_BY_ORDER[p]

# Derivative matrix
D = f_dgsem.D_matrix(p)
# Mass matrix
M = 0.5 * np.diag(lobatto_weights)
# Eigen-values and -vectors
psi = f_dgsem.eigen_L_analytical(D)
R = f_dgsem.R_matrix(D, psi)
#temp arrays for numpy inversion
I = np.eye(p + 1)
Omega = lobatto_weights.reshape((p + 1, 1))
IOmega = np.kron(I, Omega)
OmegaI = np.kron(Omega, I)
VvT = np.transpose(
    np.concatenate(
        (np.kron(I, np.ones((p + 1, 1))), np.kron(np.ones((p + 1, 1)), I)), axis=1
    )
)
#temp arrays for analytical inversion
invR = np.linalg.inv(R)
invMR = f_dgsem.diagonal_matrix_multiply(np.reciprocal(np.diag(M)), R)
invROmega = np.dot(invR, Omega) 
R2d = np.kron(R, R)
iR2d = np.kron(invR, invR)
iMR2d = np.kron(invMR, invMR)
invRROmega = np.kron(invR, invROmega)
invROmegaR = np.kron(invROmega, invR)

### Eigenvalues

Reference `numpy` performances:

In [7]:
%%timeit -r 100
f_dgsem.eigen_L_numpy(D)

31.9 µs ± 1.27 µs per loop (mean ± std. dev. of 100 runs, 10,000 loops each)


Proposed formula performances:

In [8]:
%%timeit -r 100
f_dgsem.eigen_L_analytical(D)

70.7 µs ± 2.03 µs per loop (mean ± std. dev. of 100 runs, 10,000 loops each)


### 2D matrix inversion

Reference `numpy` performances:

In [8]:
%%timeit -r 100
f_dgsem.L2d_inversion_numpy(D, M, lambda_x, lambda_y)

93.6 µs ± 1.65 µs per loop (mean ± std. dev. of 100 runs, 10,000 loops each)


Proposed formula performances:

In [7]:
%%timeit -r 100
f_dgsem.L2d_inversion_analytical(D, M, lambda_x, lambda_y)

243 µs ± 6.71 µs per loop (mean ± std. dev. of 100 runs, 1,000 loops each)


Retry by reusing matrices that can be stashed

In [7]:
%%timeit -r 100
f_dgsem.L2d_inversion_analytical(D, M, lambda_x, lambda_y, psi, iR2d, iMR2d)

14.1 µs ± 727 ns per loop (mean ± std. dev. of 100 runs, 100,000 loops each)


### 2D matrix with graph viscosity inversion

Reference `numpy` performances:

In [8]:
%%timeit -r 100
f_dgsem.L2d_inversion_viscosity_numpy(D, M, lambda_x, lambda_y, VvT, IOmega, OmegaI)

124 µs ± 3.33 µs per loop (mean ± std. dev. of 100 runs, 10,000 loops each)


Proposed formula performances:

In [7]:
%%timeit -r 100
f_dgsem.L2d_inversion_viscosity_analytical(D, M, lambda_x, lambda_y)

396 µs ± 14.6 µs per loop (mean ± std. dev. of 100 runs, 1,000 loops each)


Retry by reusing matrices that can be stashed

In [7]:
%%timeit -r 100
f_dgsem.L2d_inversion_viscosity_analytical(D, M, lambda_x, lambda_y, psi, R2d, iR2d, VvT, 
                                           invRROmega, invROmegaR)

64 µs ± 3.26 µs per loop (mean ± std. dev. of 100 runs, 10,000 loops each)
